# B.A.S.I.C. - Bank Assistant for Service, Inquiries, and Care 🏦🤖

This notebook implements a prototype of a Retrieval-Augmented Generation (RAG) system designed for customer support at a fictional bank.

**Key Features:**
- Open-source LLM (≤6B parameters)
- Retrieval from structured and unstructured sources (JSON, Excel)
- Real-time document updates
- Friendly, domain-specific, and secure answers
- Guardrails against misuse or leakage

---

### ✅ Tasks Covered in This Notebook:
1. Data Ingestion and Preprocessing
2. Embedding Creation and Indexing
3. Query-Based Retrieval
4. Response Generation via LLM
5. Simple UI with live updates
6. Architecture Diagram


```
Syed Ahsan Ullah Tanweer - 368319
Navaira Rehman - 371063
```

# Step 1: Set up Environment

We install the necessary libraries for the project.

In [7]:
!pip install -q transformers datasets faiss-cpu sentence-transformers openpyxl gradio

# Step 2: Upload Data Files

## 📁 Dataset Upload

We upload the following files:
- `bank_data.json` — contains categories/questions/answers
- `bank_info.xlsx` — Excel file with index, rate sheets, and account sheets

# Step 3: Load, Parse, Chunk and Create Embeddings of JSON + Excel

Now, let's define functions to load and parse the uploaded files.

We'll load the JSON file and parse the nested categories and questions,
and read the Excel file which may contain multiple sheets for bank products.

In [8]:
import json
import pandas as pd

# Load JSON
with open('bank_data.json', 'r') as f:
    json_data = json.load(f)

# Load Excel (you can inspect all sheet names)
excel_data = pd.ExcelFile('bank_info.xlsx')
print("Excel Sheets:", excel_data.sheet_names)

Excel Sheets: ['Main', 'Rate Sheet July 1 2024', 'LCA', 'NAA', 'NWA', 'PWRA', 'RDA', 'VPCA', 'VP-BA', 'VPBA', 'NSDA', 'PLS', 'CDA', 'NMA', 'NADA', 'NADRA', 'NUST4Car', 'ESFCA', 'NFDA', 'NSA', 'PF', 'NMC', 'NMF', 'NSF', 'NIF', 'NUF', 'NFMF', 'NFBF', 'PMYB &ALS', 'NRF', 'NHF', 'Nust Life', 'EFU Life', 'Jubilee Life ', 'HOME REMITTANCE', 'Sheet1']


#### Parse JSON Q&A Entries

In [9]:
# Flatten JSON into a DataFrame of QA pairs
qa_records = []
for cat in json_data['categories']:
    category = cat['category']
    for qa in cat['questions']:
        qa_records.append({
            'doc_id': f"json::{category}",
            'source': 'json',
            'category': category,
            'text': qa['question'],
            'answer': qa['answer']
        })

df_qa = pd.DataFrame(qa_records)
print(f"Loaded {len(df_qa)} Q&A pairs from JSON")
df_qa.head()

Loaded 15 Q&A pairs from JSON


,doc_id,source,category,text,answer
0,json::Funds Transfer / RAAST,json,Funds Transfer / RAAST,Is there a limit on the amount I can transfer ...,"Yes, 1 million is the current daily limit. Tra..."
1,json::Funds Transfer / RAAST,json,Funds Transfer / RAAST,How can I change limit of funds transfer?,"To change your funds, transfer limit, go to th..."
2,json::Funds Transfer / RAAST,json,Funds Transfer / RAAST,How can I add or update beneficiaries/ recipie...,"In the app, navigate to the “Transfer Funds” s..."
3,json::Funds Transfer / RAAST,json,Funds Transfer / RAAST,How do I perform international transactions th...,One can enable international transactions by t...
4,json::Funds Transfer / RAAST,json,Funds Transfer / RAAST,Does the app support contactless payments or d...,For contactless payments NUST Bank has a separ...


#### Parse Excel Sheets as Documents

In [10]:
# We'll treat each Excel sheet (beyond the first index sheet) as one document by concatenating all its text.
excel_docs = []
for sheet in excel_data.sheet_names[1:]:  # skip the first “index” sheet
    df_sheet = excel_data.parse(sheet).fillna('').astype(str)
    # Flatten every cell in the sheet into one long string
    text = ' '.join(df_sheet.values.flatten().tolist())
    excel_docs.append({
        'doc_id': f"excel::{sheet}",
        'source': 'excel',
        'category': sheet,
        'text': text
    })

df_excel = pd.DataFrame(excel_docs)
print(f"Loaded {len(df_excel)} documents from Excel")
df_excel.head()


Loaded 35 documents from Excel


,doc_id,source,category,text
0,excel::Rate Sheet July 1 2024,excel,Rate Sheet July 1 2024,Indicative Profit Rates\n (For Conv...
1,excel::LCA,excel,LCA,I would like to open an account with my so...
2,excel::NAA,excel,NAA,What is the Eligibility Criteria for NAA?...
3,excel::NWA,excel,NWA,Does your bank offer any accou...
4,excel::PWRA,excel,PWRA,Is there any account NUST Bank is offerin...


#### Combine All Documents into One DataFrame

In [11]:
# For JSON Q&A, combine question + answer into one field for embedding
df_qa['text'] = df_qa['text'] + " " + df_qa['answer']

# Standardize columns: doc_id, source, category, text
df_docs = pd.concat([
    df_qa[['doc_id', 'source', 'category', 'text']],
    df_excel[['doc_id', 'source', 'category', 'text']]
], ignore_index=True)

print(f"Total documents: {len(df_docs)}")
df_docs.head()

Total documents: 50


,doc_id,source,category,text
0,json::Funds Transfer / RAAST,json,Funds Transfer / RAAST,Is there a limit on the amount I can transfer ...
1,json::Funds Transfer / RAAST,json,Funds Transfer / RAAST,How can I change limit of funds transfer? To c...
2,json::Funds Transfer / RAAST,json,Funds Transfer / RAAST,How can I add or update beneficiaries/ recipie...
3,json::Funds Transfer / RAAST,json,Funds Transfer / RAAST,How do I perform international transactions th...
4,json::Funds Transfer / RAAST,json,Funds Transfer / RAAST,Does the app support contactless payments or d...


#### Preprocess Text

In [12]:
import re

def preprocess(text):
    text = text.lower()

    # Mask common PII patterns
    text = re.sub(r'\b[\w.-]+?@\w+?\.\w+?\b', '[EMAIL]', text)                      # Emails
    text = re.sub(r'\b(?:\+?\d{1,3})?[-.\s]?\(?\d{2,4}\)?[-.\s]?\d{3,5}[-.\s]?\d{4}\b', '[PHONE]', text)  # Phone numbers
    text = re.sub(r'\b\d{12,16}\b', '[CARD]', text)                                 # Credit card numbers
    text = re.sub(r'\b\d{5,}\b', '[NUM]', text)                                     # Long numerical IDs, SSNs, etc.
    text = re.sub(r'\b[A-Z]{2,}\d{5,}\b', '[ID]', text)                             # Alphanumeric IDs (e.g., AB12345)
    text = re.sub(r'\b\d{1,2}[/-]\d{1,2}[/-]\d{2,4}\b', '[DATE]', text)             # Dates (e.g. 12/04/2022)

    # Clean up whitespace and punctuation
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Apply to text column
df_docs['clean_text'] = df_docs['text'].apply(preprocess)

df_docs[['doc_id', 'clean_text']].head()

,doc_id,clean_text
0,json::Funds Transfer / RAAST,is there a limit on the amount i can transfer ...
1,json::Funds Transfer / RAAST,how can i change limit of funds transfer? to c...
2,json::Funds Transfer / RAAST,how can i add or update beneficiaries/ recipie...
3,json::Funds Transfer / RAAST,how do i perform international transactions th...
4,json::Funds Transfer / RAAST,does the app support contactless payments or d...


#### Chunk Documents for Embedding

In [13]:
import nltk
nltk.download('punkt', download_dir='/content/nltk_data')
nltk.data.path.append('/content/nltk_data')

[nltk_data] Downloading package punkt to /content/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
import pickle

# Load tokenizer from local file
with open('/content/nltk_data/tokenizers/punkt/english.pickle', 'rb') as f:
    sent_tokenizer = pickle.load(f)


In [15]:
def chunk_text(text, chunk_size=300):
    sentences = sent_tokenizer.tokenize(text)
    chunks = []
    current_chunk = ''

    for sentence in sentences:
        if len(current_chunk) + len(sentence) + 1 <= chunk_size:
            current_chunk += ' ' + sentence
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence
    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

In [16]:
chunk_records = []

for _, row in df_docs.iterrows():
    chunks = chunk_text(row['clean_text'])
    for idx, chunk in enumerate(chunks):
        chunk_records.append({
            'doc_id': row['doc_id'],
            'chunk_id': f"{row['doc_id']}::chunk_{idx}",
            'chunk_text': chunk
        })

df_chunks = pd.DataFrame(chunk_records)
df_chunks.head()

,doc_id,chunk_id,chunk_text
0,json::Funds Transfer / RAAST,json::Funds Transfer / RAAST::chunk_0,is there a limit on the amount i can transfer ...
1,json::Funds Transfer / RAAST,json::Funds Transfer / RAAST::chunk_0,how can i change limit of funds transfer? to c...
2,json::Funds Transfer / RAAST,json::Funds Transfer / RAAST::chunk_0,how can i add or update beneficiaries/ recipie...
3,json::Funds Transfer / RAAST,json::Funds Transfer / RAAST::chunk_0,how do i perform international transactions th...
4,json::Funds Transfer / RAAST,json::Funds Transfer / RAAST::chunk_0,does the app support contactless payments or d...


#### Embed the chunk_texts using sentence-transformers

In [17]:
!pip install -U sentence-transformers

In [18]:
from sentence_transformers import SentenceTransformer

# Load the model (cached in Colab after first use)
emodel = SentenceTransformer('all-MiniLM-L6-v2')

# Convert to list of texts
chunk_texts = df_chunks['chunk_text'].tolist()

# Compute embeddings (batch processing included)
embeddings = emodel.encode(chunk_texts, show_progress_bar=True, convert_to_numpy=True)

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

In [19]:
import numpy as np

# Save embeddings alongside chunk IDs
df_chunks['embedding'] = list(embeddings)
df_chunks.head()


,doc_id,chunk_id,chunk_text,embedding
0,json::Funds Transfer / RAAST,json::Funds Transfer / RAAST::chunk_0,is there a limit on the amount i can transfer ...,"[0.08316735, -0.043008957, -0.020421004, -0.06..."
1,json::Funds Transfer / RAAST,json::Funds Transfer / RAAST::chunk_0,how can i change limit of funds transfer? to c...,"[0.059385482, -0.047469538, -0.026310101, 0.03..."
2,json::Funds Transfer / RAAST,json::Funds Transfer / RAAST::chunk_0,how can i add or update beneficiaries/ recipie...,"[0.000631428, -0.054268166, -0.020424908, -0.0..."
3,json::Funds Transfer / RAAST,json::Funds Transfer / RAAST::chunk_0,how do i perform international transactions th...,"[-0.008481022, 0.0036796199, -0.0200054, -0.09..."
4,json::Funds Transfer / RAAST,json::Funds Transfer / RAAST::chunk_0,does the app support contactless payments or d...,"[-0.06960793, 0.018657116, -0.0321403, -0.1022..."


# Step 4: Retriever

#### Build a FAISS index for fast retrieval

In [20]:
!pip install faiss-cpu
import faiss

In [21]:
# Convert list of embeddings to numpy array
embedding_matrix = np.vstack(df_chunks['embedding'].values).astype('float32')

# Dimension of embeddings
embedding_dim = embedding_matrix.shape[1]

# Create FAISS index (L2 distance, flat index)
index = faiss.IndexFlatL2(embedding_dim)
index.add(embedding_matrix)

print(f"Index size: {index.ntotal}")

Index size: 327


In [22]:
def search(query, top_k=5):
    query_embedding = emodel.encode([query], convert_to_numpy=True).astype('float32')
    D, I = index.search(query_embedding, top_k)
    results = df_chunks.iloc[I[0]][['doc_id', 'chunk_id', 'chunk_text']]
    results['distance'] = D[0]
    return results

In [23]:
query = "How can I open a new bank account?"
results = search(query, top_k=5)

for i, row in results.iterrows():
    print(f"\n📄 Doc: {row['doc_id']} | Chunk #{row['chunk_id']} | Distance: {row['distance']:.4f}")
    print(row['chunk_text'])


📄 Doc: excel::VP-BA | Chunk #excel::VP-BA::chunk_0 | Distance: 0.5034
i would like to inquire about opening a current account for business with your bank. please tell me what options i have?

📄 Doc: excel::VPCA | Chunk #excel::VPCA::chunk_0 | Distance: 0.6219
i would like to inquire about opening a current account for individuals with your bank. please tell me what options i have ? nust value plus current account is specially designed for individuals to cater their financial needs.

📄 Doc: excel::NADRA | Chunk #excel::NADRA::chunk_0 | Distance: 0.7980
i would like to inquire about opening of a digital remittance account, please tell me what options are available?

📄 Doc: excel::VPBA | Chunk #excel::VPBA::chunk_0 | Distance: 0.8253
i would like to inquire about opening a current account for business purpose with your bank. please tell me what options i have? nust value premium business account has been designed to better meet the specialized banking needs of businesses that continue to